## Simulation of Ising model

(3 marks) outline the stages of your code development;  
(4 marks) sketch the dependencies between different components of your code (i.e. no spaghetti code!);  
(4 marks) a Gantt chart showing completed and ongoing code development;  
(4 marks) list completed codes in the Appendix; and  
(5 marks) report preliminary results.

## Outline the stages of your code development

* Create a 1d model as a test
    * Create a 1d model
    * Create a 1d Metropolis algorithm
    * Create a way to visualise the 1d model
    * Create a way to animate the 1d model
* Create a 2d model
    * Create a 2d model
    * Create a 2d Metropolis algorithm
    * Create a way to visualise the 2d model
    * Create a way to animate the 2d model
    * Create a graph of energy, magnetization and heat capacity against temperature for certain sizes of the system. 
    * Run the simulation multiple times and take the average of the results to reduce statistical fluctuations.
    * Determine the critical temperature using the simulation
* Create a 3d model
    * Do the same stuff as the 2d model.

## Introduction

All materials are made up of atoms. In some materials, these atoms each have individual magnetic dipoles pointing in some direction. Regions of atoms that have magnetic dipoles pointing in the same direction will have an overall magenetic dipole pointing in that same direction. And when all these regions in the material have their magnetic dipoles aligned, the material is said to be magentised. 

However, as the temperature is raised, the magnetic dipoles of the system become misaligned and the magnetisation of the system decreases. At some critical temperature, called the _Curie temperature_ The system goes through a _phase transition_ in which it loses its magnetisation. 

The Ising model is a model of a statistical mechanics system which can exhibit phase transition. We will be exploring how Ising model systems can be simulated by applying the Metropolis algorithm. 

## What is the Ising model?

Consider a 2 dimensional lattice of spins, $S_i$ in which the spins can either point up, $S_i = +1$, or point down, $S_i = -1$. Each spin interacts with its nearest neighbour through the following potential,

\begin{equation}
V_i = -Js_is_{i+1}
\end{equation}

Where $V_i$ is the potential and $J$ is the exchange energy. $J$ is a measure of the strength of the interaction. We have also set the external magnetic field to $0$ for simplicity.

The overall energy of this system is the sum of the potential $V$ over all the spins of the system. This can be seen through the following,

\begin{equation}
E = -J \sum_{i=1}^{N-1}s_is_{i+1}
\end{equation}

To calculate the magentisation, the average value of the spins can be taken. 

\begin{equation}
M = \frac{1}{N} \sum_{i = 1}^{N} s_i
\end{equation}

According to the Boltzmann distribution, the probabilty of observing any given configuration at equilibrium is given by,

\begin{equation}
P(\text{state}) = \exp(-\frac{E}{k_b T}) = \exp(-{E}{\beta})
\end{equation}

Where $k_b$ is the Boltzmann constant and $T$ is the temperature and $\beta = \frac{1}{k_b T}$. 

If $J>0$ and the temperature is low enough, the ground state will be a ferromagnet and all the spins will be aligned.  
But if $J<0$ and the temperature is low enough, the ground state will be an antiferromagnet in which all the spins are alternating.

Another way to analyse the system is to look at the internal energy and heat capacity of the system. 

## Metropolis algorithm

The steps to the metropolis algorithm applied to the Ising model is as follows,

1. We first generate a trial state by choosing one spin at random and flipping it.
2. Compute the energy of the initial state, $E_{initial}$, and the trial state, $E_{trial}$. We can also calculate their difference, $\Delta E = E_{trial} - E_{initial}$
3. Now we accept or reject the trial state according to the following rules:
    1. If $\Delta E \leq 0$, the trial state is accepted. 
    2. If $\Delta E > 0$, the trial state is accepted with a probability of $e^{\frac{\Delta E}{k_b T}}$.

We note that the Boltzmann distribution allows a state to be in a higher energy state than the ground state, but it is less likely for that to happen. The Metropolis algorithm randomly changes the individual spins such that on average, the probability of a configuration occuring follows the Boltzmann distribution. So we are able to use the Metropolis algorithm to reproduce the Boltzmann distribution.

In [ ]:
import numpy as np
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import matplotlib.animation as animation
%matplotlib notebook

First, we will create a simulation of the 1D ising model as a test,

There are N magnetic dipoles fixed in place in a line.

In [ ]:
#Inistialisation of variables
N = 10
kbt = 1
J = 1
x1d = np.subtract(np.multiply(np.random.randint((2), size=(N)),2),1)

In [ ]:
def E_1d(x1d,J):
    N = x1d.shape[0]
    result = 0
    for i in range(N-1):
            result += -J*x1d[i]*x1d[i+1]
#             print(i,i+1)
    return result

def Mag_1d(x1d):
    return np.mean(x1d)

# print(E_1d(x1d,J))
# print(Mag_1d(x1d))

In [ ]:
def metropolis1d(x1d,kbt=1,J=1):
    i = np.random.randint(len(x1d))
    initial = 0 
    trial = 0

    #initial energy calculated
    if i != len(x1d)-1:
        initial += -J*x1d[i]*x1d[i+1]
    if i != 0:
        initial += -J*x1d[i]*x1d[i-1]

    #flip the spins
    if x1d[i] == 1:
        x1d[i] = -1
    else:
        x1d[i] = 1

    #trial energy calculated
    if i != len(x1d)-1:
        initial += -J*x1d[i]*x1d[i+1]
    if i != 0:
        initial += -J*x1d[i]*x1d[i-1]

    #calculate energy difference
    delta = trial - initial

    #accept or reject the trial configuration
    #if delta is less than or equals to zero, accept trial
    if delta <= 0: 
        return x1d
    else:
        #generate a random number between 0 and 1. 
        prob = np.exp(-(delta/kbt))
        #if number is less than exp(delta/kbt), accept trial.
        if prob >= np.random.random():
            return x1d
        else:
            #else flip back and trial is rejected
            if x1d[i] == 1:
                x1d[i] = -1
            else:
                x1d[i] = 1
            return x1d

Visualise the model

In [ ]:
fig, ax = plt.subplots()
x2 = np.vstack((x1d,x1d))
cmap = colors.ListedColormap(['black','white'])
bounds=[-1,0,1]
norm = colors.BoundaryNorm(bounds, cmap.N)

img = plt.imshow(x2,interpolation='nearest',
                    cmap = cmap,norm=norm)
plt.colorbar(img,cmap=cmap,
                norm=norm,boundaries=bounds,ticks=[-1,1])

plt.show()

In [ ]:
# fig, ax = plt.subplots()

fig = plt.figure()
ax = plt.axes()
line, = ax.plot(range(len(x1d)),x1d,'ro')
time_text = ax.text(.0,1.1, "Frames passed = {}".format(0), transform=ax.transAxes)

# initialization function: plot the background of each frame
def init():
    line.set_data([], [])
    return line,

# animation function.  This is called sequentially
def animate(i):
    metropolis1d(x1d,kbt,J)
    x = np.arange(len(x1d))
    y = x1d
    line.set_data(x, y)
    time_text.set_text("Frames passed = {}".format(i))
    return line,

ani = animation.FuncAnimation(
    fig, animate, init_func=init, frames = 100, interval=200, blit=True, save_count=50, repeat = False)

plt.show()


Next is to move on to 2D ising model.
1. Create another set of $N\times N$ magnetic dipoles
2. Create another way to calculate their energy
3. Create another metropolis algorithm

In [ ]:
#initialisation
N = 20
kbt = 1.0
J = 1
x2d = np.subtract(np.multiply(np.random.randint((2), size=((N,N))),2),1)
# x=np.arange(9)
# x = np.reshape(x,(3,3))
# print("x2d is \n",x2d)
print(10*N*N)

In [ ]:
#calculate the energy for this configuration of x
def E_2d(x2d,J):
    shape = np.shape(x2d)
    result = 0
    for i in range(shape[0]):
        for j in range(shape[1]-1):
            result += -J*x2d[i,j]*x2d[i,j+1]
    for i in range(shape[0]-1):
        for j in range(shape[1]):
            result += -J*x2d[i,j]*x2d[i+1,j]
    return result

def Mag_2d(x2d):
    return np.mean(x2d)

# print(Mag_2d(x2d))
# print(E_2d(x2d,J))

In [ ]:
def metropolis2d(x2d,kbt=1,J=1):
    N = np.shape(x2d)[0]
    row = np.random.randint(N)
    col = np.random.randint(N)
    initial = 0 
    trial = 0

    #initial energy calculated
    if row != N-1:
        initial += -J*x2d[row,col]*x2d[row+1,col]
    if row != 0:
        initial += -J*x2d[row,col]*x2d[row-1,col]
    if col != N-1:
        initial += -J*x2d[row,col]*x2d[row,col+1]
    if col != 0:
        initial += -J*x2d[row,col]*x2d[row,col-1]

    #flip the spins
    if x2d[row,col] == 1:
        x2d[row,col] = -1
    else:
        x2d[row,col] = 1

    #trial energy calculated
    if row != N-1:
        trial += -J*x2d[row,col]*x2d[row+1,col]
    if row != 0:
        trial += -J*x2d[row,col]*x2d[row-1,col]
    if col != N-1:
        trial += -J*x2d[row,col]*x2d[row,col+1]
    if col != 0:
        trial += -J*x2d[row,col]*x2d[row,col-1]

    #calculate energy difference
    delta = trial - initial

    #if delta is less than or equals to zero, accept trial
    if delta <= 0: 
        return x2d
    else:
        #generate a random number between 0 and 1. 
        prob = np.exp(-(delta/kbt))
        #if number is less than exp(delta/kbt), accept trial.
        if prob >= np.random.random():
            return x2d
        else:
            #else flip back and trial is rejected
            if x2d[row,col] == 1:
                x2d[row,col] = -1
            else:
                x2d[row,col] = 1
            return x2d


In [ ]:
#visualisation

fig, ax = plt.subplots()
x2dcopy = x2d.copy()
cmap = colors.ListedColormap(['black','white'])
bounds=[-1,0,1]
norm = colors.BoundaryNorm(bounds, cmap.N)

img = plt.imshow(x2dcopy,interpolation='nearest',
                    cmap = cmap,norm=norm)
plt.colorbar(img,cmap=cmap,
                norm=norm,boundaries=bounds,ticks=[-1,1])
plt.show()

In [ ]:
#animation

fig, ax = plt.subplots()

cmap = colors.ListedColormap(['black','white'])
bounds=[-1,0,1]
norm = colors.BoundaryNorm(bounds, cmap.N)
mymap = plt.imshow(x2d,interpolation='nearest',cmap=cmap,norm=norm)
plt.colorbar(mymap,cmap=cmap,norm=norm,boundaries=bounds,ticks=[-1,1])
energy_text = ax.text(.0,1.05, "Energy = {}".format(E_2d(x2d,J)), transform=ax.transAxes)
time_text = ax.text(.0,1.1, "Frames passed = {}".format(0), transform=ax.transAxes)
mag_text = ax.text(.65,1.1, "Magnetisation = {}".format(Mag_2d(x2d)), transform=ax.transAxes)
# plt.show()

def init():  # only required for blitting to give a clean slate.
    mymap.set_array(x2d)
    return mymap

def animate(i):
    metropolis2d(x2d,kbt,J)
    mymap.set_array(x2d)
    energy_text.set_text("Energy = {}".format(E_2d(x2d,J)))
    time_text.set_text("Frames passed = {}".format(i))
    mag_text.set_text("Magnetisation = {}".format(Mag_2d(x2d)))
    return mymap

ani = animation.FuncAnimation(
    fig, animate, init_func=init, frames = 100, interval=1, blit=True, save_count=50, repeat = False)

In [ ]:
#Simulate the model for a certain length of time
for i in range(10*N*N):
    metropolis2d(x2d,kbt,J)

![Some sort of... Superweapon!](https://user-images.githubusercontent.com/38517145/76582379-091d8380-6511-11ea-965e-2b4ec1167b27.png)

#### Appendix

Completed codes:
* Create a 1d model
* Create a 1d Metropolis algorithm
* Create a way to visualise the 1d model
* Create a way to animate the 1d model
* Create a 2d model
* Create a 2d Metropolis algorithm
* Create a way to visualise the 2d model
* Create a way to animate the 2d model

on-going code development:  
for 2d model:
* Create a graph of energy, magnetisation and heat capacity against temperature for certain sizes of the system. 
* Run the simulation multiple times and take the average of the results to reduce statistical fluctuations.
* Determine the critical temperature using the simulation  

planned developments:
for 3d model:
* Create a 3d model
* Create a 3d Metropolis algorithm
* Create a way to visualise the 2d model
* Create a way to animate the 2d model
* Create a graph of energy, magnetization and heat capacity against temperature for certain sizes of the system. 
* Run the simulation multiple times and take the average of the results to reduce statistical fluctuations.
* Determine the critical temperature using the simulation 

##### References

Landau RH, José Páez Mejía Manuel, Bordeianu CC. Computational physics: problem solving with Python. Weinheim: Wiley-VCH.; 2015.